In [11]:
!pip3 install python-docx



Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 239 kB 2.7 MB/s eta 0:00:01
     |████████████████████████████████| 8.5 MB 26.9 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [10]:
from googleapiclient.discovery import build
import json

# Replace the following with your API key and playlist ID
api_key = ''
playlist_id = ''

youtube = build('youtube', 'v3', developerKey=api_key)

def get_playlist_videos(playlist_id):
    videos = []
    video_ids_seen = set()  # To keep track of video IDs already seen
    next_page_token = None

    while True:
        pl_request = youtube.playlistItems().list(
            part='snippet',
            playlistId=playlist_id,
            maxResults=50,
            pageToken=next_page_token
        )

        pl_response = pl_request.execute()

        for item in pl_response['items']:
            video_id = item['snippet']['resourceId']['videoId']
            title = item['snippet']['title']
            url = f'https://www.youtube.com/watch?v={video_id}'

            if video_id not in video_ids_seen:
                video_ids_seen.add(video_id)
                videos.append({"title": title, "url": url})
        
        next_page_token = pl_response.get('nextPageToken')

        if not next_page_token:
            break

    return videos

unique_videos = get_playlist_videos(playlist_id)

# Saving the unique videos to a JSON file
with open('unique_videos.json', 'w') as json_file:
    json.dump(unique_videos, json_file, indent=4)

print(f"Exported {len(unique_videos)} unique videos to unique_videos.json.")


Exported 123 unique videos to unique_videos.json.


# run this one, mostly

In [2]:
import json
import os
import random
from docx import Document

# Replace this with the path to your 'ad_summaries' directory
ad_summaries_dir = '/Users/mtbutler/Desktop/ad_summaries'

# Function to read the text from a Word document
def read_docx(file_path):
    doc = Document(file_path)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return '\n'.join(fullText)

# Function to generate embed code for a YouTube video
def generate_embed_code(url):
    video_id = url.split("v=")[1].split("&")[0]  # Extract video ID and handle URLs with additional parameters
    embed_code = f'<iframe class="youtube-video" src="https://www.youtube.com/embed/{video_id}" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>'
    return embed_code

# Load the existing JSON data
with open('unique_videos.json', 'r') as json_file:
    videos = json.load(json_file)

# Function to adjust the directory name based on the title
def adjust_dir_name(title):
    adjusted_title = title.replace(" ", "-")
    if adjusted_title.endswith("-wm"):
        adjusted_title = adjusted_title[:-3]  # Remove the last 3 characters ("-wm")
    return adjusted_title

# Iterate through each video in the JSON file
for video in videos:
    title = video["title"]
    # Adjust the directory name based on the title
    dir_name = adjust_dir_name(title)
    full_dir_path = os.path.join(ad_summaries_dir, dir_name)
    
    # Initialize summaries in video object
    video['A'] = ''
    video['B'] = ''
    video['selection'] = {'A': '', 'B': ''}
    video['embed'] = generate_embed_code(video['url'])  # Add embed code using the video's URL

    # Check if the directory exists
    if os.path.exists(full_dir_path):
        # List all docx files in the directory
        docx_files = [f for f in os.listdir(full_dir_path) if f.endswith('.docx')]
        if docx_files:
            # Sort files to ensure the last one is the 'computer' version
            docx_files.sort()
            computer_version = docx_files[-1]  # The computer version is the last file
            human_versions = docx_files[:-1]  # All other files are human versions

            # Select a random human version
            if human_versions:
                random_human_version = random.choice(human_versions)
                selected_files = [computer_version, random_human_version]

                # Randomize the order of selected files into A or B
                random.shuffle(selected_files)
                video['A'] = read_docx(os.path.join(full_dir_path, selected_files[0]))
                video['B'] = read_docx(os.path.join(full_dir_path, selected_files[1]))

                # Track which version is selected
                video['selection']['A'] = selected_files[0]
                video['selection']['B'] = selected_files[1]

# Write the updated data back to the JSON file
with open('unique_videos_updated-withGPT.json', 'w') as json_file:
    json.dump(videos, json_file, indent=4)

print("Updated summaries and embed codes have been added to unique_videos_updated.json.")


Updated summaries and embed codes have been added to unique_videos_updated.json.


In [1]:
import json
import random

# Step 1: Read the JSON data from unique_videos_updated.json
with open('unique_videos_updated.json', 'r') as file:
    data = json.load(file)

updated_data = []

for item in data:
    # Filter summaries to include only non-empty ones and pair them with their number (1, 2, or 3)
    summaries = [(key[-1], value) for key, value in item.items() if key.startswith('summary') and value]
    # Randomly select two summaries, duplicate if necessary
    selected_summaries = random.sample(summaries, 2) if len(summaries) > 1 else summaries * 2

    # Construct the new object format
    new_item = {
        "title": item["title"],
        "url": item["url"],
        "A": selected_summaries[0][1],
        "B": selected_summaries[1][1],
        "selection": [int(selected_summaries[0][0]), int(selected_summaries[1][0])]
    }
    updated_data.append(new_item)

# Step 3: (Optional) Write the processed data back to a new JSON file
with open('unique_videos_processed.json', 'w') as outfile:
    json.dump(updated_data, outfile, indent=4)

print("Processing complete. Data saved to 'unique_videos_processed.json'.")


Processing complete. Data saved to 'unique_videos_processed.json'.


In [2]:
import csv
import json

# Update these paths to the actual locations of your files
json_file_path = 'unique_videos_processed.json'
csv_file_path = 'videos_for_qualtrics.csv'

# Function to safely encode text for CSV, including converting lists to strings
def encode_for_csv(text):
    if isinstance(text, list):  # Convert list to string for 'selection'
        return ';'.join(str(num) for num in text)
    return text.replace('"', '""')  # Escape double quotes by doubling them

# Function to generate iframe embed code from YouTube URL
def generate_embed_code(url):
    video_id = url.split("v=")[1].split("&")[0]  # Extract video ID and handle URLs with additional parameters
    embed_code = f'<iframe width="560" height="315" src="https://www.youtube.com/embed/{video_id}" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>'
    return embed_code

# Read the JSON data
with open(json_file_path, 'r', encoding='utf-8') as json_file:
    json_data = json.load(json_file)

# Write the modified data to a CSV file
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file, quoting=csv.QUOTE_ALL)  # Use QUOTE_ALL to handle special characters in text
    # Write the header row
    writer.writerow(['Title', 'EmbedCode', 'Summary_A', 'Summary_B', 'Selection'])

    for video in json_data:
        # Generate iframe embed code for each video URL
        embed_code = generate_embed_code(video.get('url', ''))
        
        writer.writerow([
            encode_for_csv(video.get('title', '')),
            encode_for_csv(embed_code),  # Save the iframe embed code instead of the URL
            encode_for_csv(video.get('A', '')),
            encode_for_csv(video.get('B', '')),
            encode_for_csv(video.get('selection', ''))
        ])


In [3]:
import json

def get_video_id(youtube_url):
    """
    Extracts the video ID from a YouTube URL.
    """
    if 'youtu.be' in youtube_url:
        return youtube_url.split('/')[-1]
    if 'youtube.com' in youtube_url:
        return youtube_url.split('v=')[-1].split('&')[0]
    return None

# Step 1: Read the JSON data
with open('unique_videos_processed.json', 'r') as file:
    data = json.load(file)

for item in data:
    # Step 2: Extract the YouTube video ID from the URL
    video_id = get_video_id(item['url'])
    
    # Step 3: Construct the embed code
    if video_id:
        embed_code = f'<iframe width="560" height="315" src="https://www.youtube.com/embed/{video_id}" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>'
        # Step 4: Add the embed code to the item
        item['embed'] = embed_code

# Step 5: Write the modified data to a new file
with open('unique_videos_embed.json', 'w') as outfile:
    json.dump(data, outfile, indent=4)

print("Embed codes added. Data saved to 'unique_videos_embed.json'.")


Embed codes added. Data saved to 'unique_videos_embed.json'.


In [1]:
video_data = [
    {
        "title": "P 1500 75016",
        "url": "https://www.youtube.com/watch?v=--BAeCHBh18",
        "A": "The ad features a cartoon with different types of individuals endorsing the republican candidate and stating their dislike for the Democratic nominees. The ad emphasizes that everyone likes the candidate and that the people will take him to Washington; therefore, now is the time to vote for the candidate.",
        "B": "The advertisement sings along to a tune of \"Ike for president.\" Cartoon Americans and patriotic images stress the concept that everyone supports this candidate and for what he stands. It is expressed that bringing the candidate to Washington would be an American duty that everyone is fulfilling.",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/--BAeCHBh18\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1436 67065",
        "url": "https://www.youtube.com/watch?v=-DCuS2T_iyY",
        "A": "The ad criticizes the candidate for denying that he aided in engineering a 100 billion dollar tax increase. The ad cites a news source that confirms the candidate's role in the budget deal and urges the public to see that the candidate only flipped on the deal to cover himself politically. ",
        "B": "The opposing candidate denies having helped to create a $100 billion tax increase, but his significant role in it has been proved by news sources. The opposing candidate then switched his position and voted against the bill. The opponent is negatively characterized as a \"Washington politician.\"",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/-DCuS2T_iyY\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1425 66916",
        "url": "https://www.youtube.com/watch?v=0rhBETx6jgM",
        "A": "A narrator urges the audience to listen to the candidate's message, suggesting it may align with their own beliefs. The candidate then advocates for a robust and strengthened social security system. A slogan affirming the first message of the ad reads: \"In your heart, you know he's right.\"",
        "B": "The advertisement tells viewers to listen and understand that the candidate shares their opinions. The candidate wants a sound and stronger Social Security system. He also wants the dollar to have real purchasing power. It is affirmed that the candidate is right and advised that viewers vote for him.",
        "selection": [
            3,
            2
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/0rhBETx6jgM\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 2116 141528",
        "url": "https://www.youtube.com/watch?v=1Iu7nao_Ev4",
        "A": "The opponent is asked his position on women's access to birth control. The advertisement displays information about a related Supreme Court case affirming that protection. The candidate answers that he thinks the issue should be returned to the states, and the candidate is framed as harmful for women.",
        "B": "The ad discusses the candidate's opinion on women's access to birth control. The ad features the candidate's ignorance of the concept by bringing legal precedent up. Thus, the ad emphasizes that because of such views on contraception and women's health, the candidate is wrong for America's women.",
        "selection": [
            1,
            2
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/1Iu7nao_Ev4\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1905 108595",
        "url": "https://www.youtube.com/watch?v=1ZNuDunUAMQ",
        "A": "The ad discusses that the people of Florida face an important choice regarding how to elect a president who will fight for them, their family, their income, and control spending. It contrasts the candidate with the opposing candidate and argues that the opposing candidate will fail to achieve this for them.",
        "B": "A former governor advises Florida voters in the presidential election. He compares the candidate with one that will fight for working families, cut spending, and eliminate waste, instead of one who will raise taxes and create a deeper debt. The candidate has the experience and bipartisanship necessary for tough times.",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/1ZNuDunUAMQ\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 2098 140422",
        "url": "https://www.youtube.com/watch?v=4l7lH90vGhw",
        "A": "The ad portrays that the candidate's plan will take money out of the pockets of the people to give tax breaks to billionaires. The ad urges voter to keep his hands out of their pockets by not voting for him and realizing that he is only running for president for the top 1%.",
        "B": "The advertisement depicts the opposing candidate as trying to steal from the pockets of members of the working class. This \"stolen\" money would be used to pay for tax breaks for millionaires. The opponent is running to be president for the top 1% instead of the rest of America.",
        "selection": [
            1,
            2
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/4l7lH90vGhw\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1415 68241",
        "url": "https://www.youtube.com/watch?v=5qm69xeMyX0",
        "A": "The ad discusses the president and his welfare reform, emphasizing his signing of welfare reform, creating jobs through tax incentives, and offering job training for all workers. The candidate is highlighted as someone who can meet America's challenges and move the country forward.",
        "B": " The current president ensures responsibility from those receiving welfare. His welfare reform included work requirements, time limits, and teenage mothers staying in school. His next plans include creating jobs, providing tax incentives, and job training, which the opponents tried to cut. The candidate can move America from welfare to work.",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/5qm69xeMyX0\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1905 108583",
        "url": "https://www.youtube.com/watch?v=9S9PSmeTSsE",
        "A": "The ad critiques the candidate for representing the wrong kind of change for America and for being friends with many concerning individuals. The ad attacks the candidate's pastor for being racist, the candidate's political alliances, and more. It questions the type of change the candidate wants for America. ",
        "B": "The opposing candidate is described as being the wrong change for Michigan and America. He has alleged negative associations with a domestic terrorist, a racist anti-American pastor, and a disgraced Michigan mayor. The advertisement explains that this is not the kind of change that should be brought to America.",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/9S9PSmeTSsE\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 2125 142151",
        "url": "https://www.youtube.com/watch?v=Cn7gsZUVnkg",
        "A": "The advertisement focuses on the opposing candidate and the importance of his performance in the debate against the incumbent president. This debate will make or break his race against the candidate. While there are high expectations for the opposing candidate in this debate, his stakes in this are very high.",
        "B": "The ad features multiple discussions about the candidate and the importance of debates. These individuals discuss the candidate and how well he needs to do in the debates to get the support of the people. They all have great expectations for the candidate and expect him to come out of the debates successfully. ",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/Cn7gsZUVnkg\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1353 63367",
        "url": "https://www.youtube.com/watch?v=DKXW68CYpKI",
        "A": "The ad criticizes the opposing candidate for supporting an untested radical plan that would end Social Security. It then labels his platform as untested leadership with risky ideas such as the social security plan, and urges voters not to vote for him.",
        "B": "The advertisement explains that the opposing candidate has radical ideas to end Social Security. This is an untested policy which would be a problem for Americans. The candidate's untested leadership and risky ideas are portrayed as negative and are examples of why this opposing candidate should not win.",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/DKXW68CYpKI\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1908 108808",
        "url": "https://www.youtube.com/watch?v=DSwz6KRzBMU",
        "A": "The candidate discusses his strong values and family. He explains his history and how he learned accountability, self-reliance, and hard work. He emphasizes that these values have driven him to work for the people and pass laws to cut taxes, move people from welfare to work, and more. He promises to never forget these values.",
        "B": "The candidate connects with America's strong families and values. Raised by his single mother and grandparents, he was taught accountability, hard work, and self-reliance. He took jobs and loans to get through college. As a senator, he cut taxes and helped working families. He would remember these values as president. ",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/DSwz6KRzBMU\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1617 81835",
        "url": "https://www.youtube.com/watch?v=E4UYPUQUw10",
        "A": "The ad features an introduction of a campaign speech that emphasizes that the next president must be someone with a vision, integrity, and experience to lead us. The candidate then speaks and advocates his two most important beliefs: tax cuts and education for all American Children. ",
        "B": "The advertisement shows a primary campaign speech in New Hampshire at which the candidate is supported as the next president. He is described as having the executive experience necessary to lead America. His two most important issues are tax cuts and child education. This is said to be continued.",
        "selection": [
            1,
            2
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/E4UYPUQUw10\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1817 102732",
        "url": "https://www.youtube.com/watch?v=EVTDp8KnOTE",
        "A": "The ad attacks a candidate for being indecisive and changing his mind on important issues such as Iraq, taxes, welfare reform, and more. The ad urges viewers to think about whether they can trust him and whether they should vote for him.",
        "B": "The advertisement outlines an analogy with a man whose indecision in life decisions unfolds into chaotic events. Like this man, the candidate has been indecisive about very important policies. Since the candidate has changed his mind on all of these issues, he cannot be trusted as president.",
        "selection": [
            1,
            2
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/EVTDp8KnOTE\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1436 67079",
        "url": "https://www.youtube.com/watch?v=FcGCDeUruVs",
        "A": "The candidate explains that a flat tax and a family tax cut are good ideas for Americans. The advertisement details examples of what a family and retired couple would save with a flat tax. The candidate argues that the flat tax is pro-family, and he would support this as president.",
        "B": "The ad endorses the candidate's flat tax that would provide a family tax cut, going on to provide an example of how much a family would save. The candidate emphasizes that the flat tax would strengthen families and be simple and honest, a major change for Washington. ",
        "selection": [
            1,
            2
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/FcGCDeUruVs\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1619 81865",
        "url": "https://www.youtube.com/watch?v=GuhnGUbcLeg",
        "A": "The candidate's mother praises him for his values. The candidate explains that his values are why he proposes to lower the flat tax by 16%, protect unborn children, and to save social security. His message serves to show that he will advance America's values.",
        "B": "Showing family values, the candidate is seated next to his proud mother and describes the important values he learned growing up in working-class Kentucky. In running for president, he prioritizes a flat tax, protecting unborn children, and saving Social Security for present and future generations.",
        "selection": [
            1,
            2
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/GuhnGUbcLeg\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1708 90169",
        "url": "https://www.youtube.com/watch?v=GxLatwkppzE",
        "A": "The ad criticizes a candidate for failing to meet with the head of the Soviet Union. The candidate being endorsed emphasizes that he will push for annual summit conferences with the Soviets, negotiate to stop the arms race, and take charge of American foreign policy to bring peace.",
        "B": "While there are 50,000 nuclear warheads in the world, the current president failed to meet with the leader of the Soviet Union. As president, the candidate would fulfill this responsibility, end the arms race, and take charge of foreign policy. He would use America's strength to maintain peace.",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/GxLatwkppzE\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1907 108725",
        "url": "https://www.youtube.com/watch?v=I8qWf7DYJQA",
        "A": "The ad features an individual expressing disappointment in the candidate as a senator. The senator is critiqued for failing to provide solutions and offering alternatives as publicity stunts.  The ad emphasizes how the people expected better and that the senator let them down.",
        "B": "A man addresses the opposing candidate with disappointment. The opponent had promised to make driving affordable, and the man had believed him. Instead, the opponent approved offshore drilling that would not fulfill his promise in the short term. The man explains that this is a gimmick, not a solution.",
        "selection": [
            1,
            2
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/I8qWf7DYJQA\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1903 108459",
        "url": "https://www.youtube.com/watch?v=Iv_rt_91YiY",
        "A": "The candidate describes his military service for America and now being dedicated to a cause greater than his own. His policy goals are to grow the economy and cut government waste. The candidate urges viewers to vote for him. Inspirational music and images of the candidate play throughout the advertisement.",
        "B": "The candidate discusses how he has served the country since a young age and continues to do so. He emphasizes his goals to grow the economy and cut government waste, urging the public to stop hoping for a strong America and to instead vote for one by voting for him.",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/Iv_rt_91YiY\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1618 81849",
        "url": "https://www.youtube.com/watch?v=KQqif36gd24",
        "A": "A diverse group of working-class people are described as the \"faces of America.\" The candidate listens to them, understands them, and shares their dreams. At rallies, he emphasizes that he would prioritize and protect Social Security. The candidate is cut out for a difficult job as president.",
        "B": "The ad portrays American workers of different occupations who pay taxes and live by the rules of the government. It highlights how the candidate hears their voices and understands their fears regarding social security and their money. The candidate's character and his hard work for the people are praised.",
        "selection": [
            1,
            2
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/KQqif36gd24\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1708 90180",
        "url": "https://www.youtube.com/watch?v=N24we0lvE2U",
        "A": "The candidate tells the country that what it is missing is a president who has a policy that puts Americans back to work. The ad emphasizes the leadership experience of the candidate such as being the Attorney General of Minnesota, a U.S. senator, and Vice President to show his ability to fight for the people.",
        "B": "The candidate explains that Americans want to work but need a president whose policies can put them back to work. He is described as having a lifetime of leadership in various political positions. As president, the candidate would fight on Americans' side and first ensure that Americans have work.",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/N24we0lvE2U\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 2123 141983",
        "url": "https://www.youtube.com/watch?v=S-zA14tY-M4",
        "A": "The advertisement addresses the opponent's claims of his friendliness toward the coal industry. However, the opponent plans to ship American jobs overseas, has Swiss bank accounts and tax havens, and aims to shut down coal power plants for harming people. The advertisement separates the opponent as \"not one of us.\"",
        "B": "The ad critiques the opposing candidate for saying he supports the coal industry and the people while he has financial irregularities that hurt Americans. The ad strives to emphasize how the candidate is not one of the people and will not serve their needs as he claims he will.",
        "selection": [
            1,
            2
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/S-zA14tY-M4\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1425 66913",
        "url": "https://www.youtube.com/watch?v=TE0g4kZn-1I",
        "A": "A woman is shown asking for the candidate's ideas on how to stop the spread of communism throughout the world. The candidate responds with plans to reform American alliances and repair the current administration's damages to NATO. He wants to work with these allies as equals to ensure peace.",
        "B": "The ad portrays a citizen who is concerned about the spread of communism and asks the candidate about what he will do to stop the spread of communism. The candidate discusses his plan to rebuild alliances, work together to rebuild peace, and to clean up the damage done by the existing administration.  ",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/TE0g4kZn-1I\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 2015 136329",
        "url": "https://www.youtube.com/watch?v=TSIEzDwfjgU",
        "A": "The ad discusses the candidate's visit to Ohio and his promise to provide Ohio with jobs. The ad then features statistics on the high number of Ohioans who are unemployed and the jobs lost since the stimulus was passed. It emphasizes to the people that Ohio needs to change direction.",
        "B": "The advertisement shows the current president (opponent) giving a speech on jobs in Ohio. The opponent and an Ohio senator promised more jobs for Ohio but have failed on this promise while spending Americans' money. Ohio voters are encouraged to change direction and vote these officials out of office.",
        "selection": [
            1,
            2
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/TSIEzDwfjgU\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 2015 136314",
        "url": "https://www.youtube.com/watch?v=Umk2PS6Zuz8",
        "A": "The advertisement shows the current president (opponent) and his stance on the economy. He describes his plans that have not gone through Congress, and he was just on vacation. In the next year, unemployment is still too high with millions out of work.  America needs jobs instead of another speech.",
        "B": "A candidate speaks on the economy and how it is not growing fast enough. He highlights that he has things before Congress that they should be moving immediately. The ad then switches to attacking the candidate for failing to make any change as unemployment is still high and millions of Americans are out of work.",
        "selection": [
            1,
            2
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/Umk2PS6Zuz8\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1615 81773",
        "url": "https://www.youtube.com/watch?v=VtZ5xaiSiik",
        "A": "The ad features two different individuals who say they agree on very little but do both agree on endorsing the candidate in question. The candidate checks all the boxes for each individual regarding gun control, Medicare, education, tax policy, and more. The ad highlights that it is not a tough choice to make.",
        "B": "The advertisement depicts an older white man and younger black man sitting in New York. While there are few things the two men agree on, they both want the candidate for president. They support his stance on Medicare, gun control, education, and tax policy. This shows his diversity in approval.",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/VtZ5xaiSiik\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1908 108788",
        "url": "https://www.youtube.com/watch?v=WZLZG9JBshc",
        "A": "The candidate's sister describes his ability to understand, relate to, and listen to others. His grandmother describes his depth and broadness of view. It is clear that he cares deeply about his daughters and family values, and he wants to ensure that every child would have their same opportunities.",
        "B": "The ad features different individuals discussing their ability to relate to the candidate and how they feel understood by him. The candidate is praised for his depth and ability to listen. His family and children are discussed and connected to why he wants every child to have opportunities without barriers. ",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/WZLZG9JBshc\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1500 75019",
        "url": "https://www.youtube.com/watch?v=Wav9MpjeIMA",
        "A": "While men are standing discussing politics, another man flippantly asks what it should mean to him. The man is guided into his wallet, showing that he has more money, a Social Security card, and auto registration that he did not have 20 years ago. This is attributable to Democrats.",
        "B": "The ad explains what politics means to the people and what the democratic party, specifically, has done for them. Social security, insured money in banks, and greater pay are used as examples of how the democrats have done well for the people and how the republicans will take this away.",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/Wav9MpjeIMA\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1691 86717",
        "url": "https://www.youtube.com/watch?v=XFZfXGvukTQ",
        "A": "The advertisement argues that the opposing candidate would have a litmus test established for the Joint Chiefs of Staff, showing footage of him explaining that requirement in a debate. The opponent then falls back on what he said in an indecisive manner. It is implied that he is an unfit candidate.",
        "B": "The ad attacks a candidate for double-talking as he first says that he would make it a requirement for the Joint Chiefs of Staff that the individual support his policy. The candidate then goes back and forth on whether this was what he meant to convey or not.",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/XFZfXGvukTQ\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1436 67087",
        "url": "https://www.youtube.com/watch?v=ZPW55c6LH60",
        "A": "The ad criticizes the politicians of Washington and how they are misleading the public about the flat tax. The ad promotes the candidate's flat tax by explaining how families and wage earners pay less and that the flat tax takes power from the politicians and gives money back to the people.",
        "B": "The advertisement accuses the opposing candidates of lying to Americans about the flat tax because they are losing the election. However, under the candidate's flat tax, it is expected that everyone would pay less, not more, in taxes. This would remove power from politicians and save money for Americans. ",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/ZPW55c6LH60\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 2060 138579",
        "url": "https://www.youtube.com/watch?v=_HhVtVxkcRw",
        "A": "The opposing candidate receives backlash from his recent attack advertisement against the candidate. The advertisement lists 14 different members of the opponent's party who have called him out on this. This is described as bad for this current president, while it has been a bad term for Americans.",
        "B": "The ad critiques the president and discusses an ad he made against a candidate that many think is unfair. Further, it discusses how the president is receiving a lot of negativity as a result and many of his loyal supporters are even saying enough.",
        "selection": [
            1,
            2
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/_HhVtVxkcRw\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1581 79623",
        "url": "https://www.youtube.com/watch?v=_YD8201kOfQ",
        "A": "A mother asks the candidate why she has to spend so much on food if the government says this is the best the economy has been. The candidate blames the many taxes she must pay and emphasizes the need to cut costs which means cutting taxes. His administration will do so.",
        "B": "The candidate addresses a concern of a mother with her children about the price of eggs. The candidate explains that this is because there are over 100 taxes placed on just one egg. To lower the cost of things, taxes must be cut, which he implies he would do.",
        "selection": [
            1,
            2
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/_YD8201kOfQ\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1500 75016",
        "url": "https://www.youtube.com/watch?v=aGczMlmSVUI",
        "A": "The ad features a cartoon with different types of individuals endorsing the republican candidate and stating their dislike for the Democratic nominees. The ad emphasizes that everyone likes the candidate and that the people will take him to Washington; therefore, now is the time to vote for the candidate.",
        "B": "The advertisement sings along to a tune of \"Ike for president.\" Cartoon Americans and patriotic images stress the concept that everyone supports this candidate and for what he stands. It is expressed that bringing the candidate to Washington would be an American duty that everyone is fulfilling.",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/aGczMlmSVUI\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 2114 141382",
        "url": "https://www.youtube.com/watch?v=cui6DFTWKfs",
        "A": "The ad discusses Pennsylvania's importance in our history and that the need for a strong leader has never been greater. Many problems plague the state and nation which the candidate can help solve. The ad urges voters to elect a new president and it will be the turning point. ",
        "B": "The advertisement summarizes Pennsylvania's role in forming American history and revolutionary change. The \"American dream\" is now fading, but Pennsylvania could be a turning point with the opportunity to fix that. By electing the candidate, the state can restore belief in America. Inspirational music and themes help push this message.",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/cui6DFTWKfs\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1616 82778",
        "url": "https://www.youtube.com/watch?v=eqMAnP1qFXk",
        "A": "Eight Nobel Laureates are used as credibility in reviewing the opponent's economic plans. The combination of trillions in Social Security and tax cuts to the wealthy lead these experts to conclude that his economic plan exhausts the projected surplus, interest rates, and the deficit, which cannot be good for America.",
        "B": "The ad criticizes the candidate's election plan regarding social security by showing how Nobel Laureates evaluated the plan and found that it exhausts the projected surplus. The ad emphasizes that the plan does not add up ad it will only increase interest rates and the deficit. ",
        "selection": [
            1,
            2
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/eqMAnP1qFXk\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1818 102774",
        "url": "https://www.youtube.com/watch?v=fVWD1US6Ef4",
        "A": "The ad praises the candidate for the reforms he passed as governor that produced actual results. Further, it praises his time as president and how he passed the first significant education reform in years that greatly impacted public schools, teachers, and the education that children receive.  ",
        "B": "As governor, the candidate enacted reforms that produced dramatic changes in education. As president, the candidate signed very significant education reforms. His education priorities include accountability and high standards, allowing parents to measure their child's progress. Because of the president, schools now require raised standards, quality teachers, and parent accountability.",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/fVWD1US6Ef4\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1581 79612",
        "url": "https://www.youtube.com/watch?v=fYXWOho9WKo",
        "A": "The candidate answers a question about the extent of waste in Washington. He responds that waste is so bad that just one bureau recently lost 400 million dollars, and the FBI cannot find it. He explains that it is time for change, implying that he can bring this change.",
        "B": "The ad features an individual asking the candidate about how bad waste in Washington is. The candidate emphasizes how the government has lost 400 million and that no one can find it. He uses this to emphasize the need for change and the weakness of the current administration.",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/fYXWOho9WKo\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1816 102673",
        "url": "https://www.youtube.com/watch?v=fze9LoP1iT4",
        "A": "The ad discusses missing pages of evidence missing during the 9/11 Congressional hearing. The ad blames the candidate for hiding this and being involved with the Saudis, arguing that the candidate is too close to the Saudis and is a threat to America's well-being.",
        "B": "The congressional report on 9/11 had 28 missing pages, which allegedly confirmed that Saudi Arabia funded these terrorists. This evidence was kept secret under the current president who is friends with a Saudi Arabia defender. This shows that the current president and the Saudis are too close for America's comfort.",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/fze9LoP1iT4\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1817 102725",
        "url": "https://www.youtube.com/watch?v=gYhwyxJvknA",
        "A": "The ad discusses the many symbols that represent America and its character. It then critiques the candidate for disrespecting the symbols of America. It uses this disrespect to argue that America cannot trust someone who has disrespected the country and what it stands for.",
        "B": "The advertisement explains that symbols represent the best things about America and are meant to be respected like the heroes they represent. Some people, however, do not share this respect. The candidate renounced his country's war symbols, and now he cannot be trusted as an American leader.",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/gYhwyxJvknA\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1361 63305",
        "url": "https://www.youtube.com/watch?v=h9yEYqdALYs",
        "A": "The advertisement displays two men kissing. In the background, previous policies about the AIDS epidemic are described as unsuccessful in finding a cure. It is made clear that prevention of AIDS is not complicated, as \"kissing doesn't kill,\" and the proper use of protection will prevent the spread of AIDS.",
        "B": "The ad critiques an opposing candidate and past leaders for their failure to provide presidential leadership to prevent the spread of AIDS. The democratic candidate emphasizes his understanding of HIV/AIDS and proper precautions. He urges for action and for America to fight AIDS.",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/h9yEYqdALYs\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1631 83508",
        "url": "https://www.youtube.com/watch?v=hTpm7XZHLmY",
        "A": "The advertisement attacks the opponent's actions as governor. He gave tax breaks to oil companies, opposed child healthcare, kept minimum wage low, and did not police polluters. His state ranks last in healthcare and air pollution. His harmful plans promise trillions in Social Security and tax cuts to the wealthy.",
        "B": "A candidate's time as governor of Texas is criticized. His giving big oil a tax break, his opposing of health care for 200,000 plus kids, his leaving Texas to rank 50th in family health care and the minimum wage at an extreme low, and his lack of pollution regulation are all mentioned.",
        "selection": [
            1,
            2
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/hTpm7XZHLmY\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1827 103450",
        "url": "https://www.youtube.com/watch?v=i1lFGZf0FNE",
        "A": "The ad critiques the candidate and right-wing Republicans for helping big drug companies while harming the American public. The ad switches over to praise the supporting candidate and democratic party that will force drug companies to lower prices and offer a real plan for Americans.",
        "B": "The opponent has helped big drug companies by prohibiting seniors from getting affordable and safe prescriptions from Cananda, bailing out drug companies, and allowing prices to skyrocket. The candidate, however, would force companies to lower prices and allow low-cost medicine from Canada, pushing America into a new and better direction.",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/i1lFGZf0FNE\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1498 74079",
        "url": "https://www.youtube.com/watch?v=ilJtdEz6kus",
        "A": "The ad criticizes the candidate for making voters think he sincerely believes in the civil rights portion of the Republican platform when in reality only says this to get votes in certain states. The ad supports the democratic candidate and says he truly endorses his party's platform in the entire country.",
        "B": "The advertisement accuses the opponent of being inconsistent with his stance on civil rights and misrepresenting his party. A supporter of the opponent explains that the opponent changes his platform to fit his audience. The candidate would not do this and is firm in his support for civil rights.",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/ilJtdEz6kus\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1840 104205",
        "url": "https://www.youtube.com/watch?v=jNXk-VUdt8g",
        "A": "Repeat of ad P-1817-102732",
        "B": "The advertisement outlines an analogy with a man whose indecision in life decisions unfolds into chaotic events. Like this man, the candidate has been indecisive about very important policies. Since the candidate has changed his mind on all of these issues, he cannot be trusted as president.",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/jNXk-VUdt8g\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1411 68113",
        "url": "https://www.youtube.com/watch?v=jenHeKjoLgo",
        "A": "Parents describe being able to spend time with their sick child before she died, without losing their jobs. The current president signed the family leave law to allow this, while the opposing candidate worked for years to defeat such a measure. The candidate is shown as caring for these families.",
        "B": "The ad discusses the family and parent medical leave law signed by the president and the impact this has had on families. It then critiques the opposing candidate for leading a fight against the family leave law and emphasizes that unlike him, the president understands what families go through.",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/jenHeKjoLgo\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1772 101721",
        "url": "https://www.youtube.com/watch?v=uiy-S3Xg0M0",
        "A": "The ad utilizes two individuals to show how the insurance industry is attacking the healthcare reform plan by the candidate. The ad emphasizes that the plan would guarantee Americans private insurance, expand doctors and plans, and stop the industry from raising prices or dropping coverage but the insurance industry stands in the way.",
        "B": "The advertisement describes attacks on the candidate's healthcare reform plan that have been paid for by the insurance industry. This plan, however, would help Americans by guaranteeing private insurance, expanding choices, and preventing insurance from increasing prices and dropping coverage. Viewers are advised to remember this when viewing attack advertisements.",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/uiy-S3Xg0M0\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1415 68229",
        "url": "https://www.youtube.com/watch?v=vfQlWsKSVQ0",
        "A": "As president, the candidate cut taxes for millions of Americans, created 10 million new jobs, and wants tax credits for college graduates. The opposing candidate, however, voted to raise payroll taxes, resulting in $900 billion in higher taxes, and he will also likely cut Medicare, education, and the environment. ",
        "B": "The ad emphasizes all that the president has done for Americans while the opponent has only made it harder for Americans by making them spend billions more on taxes. The ad highlights that the candidate will have to make deep spending cuts on education, Medicare, and more. ",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/vfQlWsKSVQ0\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1631 82797",
        "url": "https://www.youtube.com/watch?v=wRIZyBatvr8",
        "A": "The ad discusses the candidate's education plan, which includes 100,000 more teachers to lower class sizes and to help middle-class families by providing a $10,000 deduction every year for college tuition. The candidate strives to make education a main priority for the country.",
        "B": "The candidate describes his policy goals in education as a priority, in comparison to his opponent. He explains his plans in education accountability, smaller classroom sizes with more teachers for better learning, and making college tuition payments tax deductible and more affordable for middle-class families.",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/wRIZyBatvr8\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1498 74101",
        "url": "https://www.youtube.com/watch?v=ybm2Y_hH2Tc",
        "A": "The candidate asks the people of Minnesota for their help in the election. He emphasizes Minnesota's need for progressive federal policy that will protect agriculture which has declined due to the past and present individuals in office. He emphasizes that he will provide the strong and progressive administration the people need.",
        "B": "The candidate explains why Minnesotans ought to vote for him. Minnesota would benefit from his progressive federal policies to protect declining agriculture, which would get worse under the opposing candidate. He calls for increased jobs, better education, and elderly medical care. His progressive campaign is what Minnesota needs.",
        "selection": [
            1,
            2
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/ybm2Y_hH2Tc\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1190 35257",
        "url": "https://www.youtube.com/watch?v=2yHBUscdRcI",
        "A": "The advertisement compares the economic ideals of the candidate and his opponent. While the candidate's economic policies include cutting taxes, cutting spending, creating jobs, and growth, the opponent's policies only include raising taxes. The candidate's plans are said to work for Americans while the opposing candidate works against Americans.",
        "B": "The ad compares the two candidates and their methods for dealing with the nation's economy. It specifically discusses taxes, spending, jobs, and growth. The ad emphasizes that the candidate's economics works for the people while the opposing candidate's economics works against the people.",
        "selection": [
            1,
            2
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/2yHBUscdRcI\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 358 9904",
        "url": "https://www.youtube.com/watch?v=9or4wTEpQ2Y",
        "A": "The advertisement shows a bomb explosion. It recounts the opposing candidate's reckless statement about nuclear weapons. This alludes to the common Cold War mentality and fear of nuclear war. Viewers are urged to vote for the candidate on Election Day, as the stakes are too high.",
        "B": "The ad utilizes what the opposing candidate said about the setting off of the nuclear bomb as merely being another weapon, to advocate voters to choose Johnson. The ad strives to utilize this comment to show how the stakes are too high in the election and that there is a lot to lose.",
        "selection": [
            1,
            3
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/9or4wTEpQ2Y\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 507 94878",
        "url": "https://www.youtube.com/watch?v=DwV8o9DzIRc",
        "A": "The ad discusses how troops have been left in countries and are slow-burning fuses to war. It emphasizes how the vice president and president agree on leaving troops as bargaining chips. The candidate is disappointed in having the sons of the country as bargaining chips and urges the people to vote for new leadership.",
        "B": "The advertisement describes the president's sending troops to Central America as \"advisors\" to the countries. Here, the troops await war while they serve as bodyguards to dictators. The vice president supports this action being done with American soldiers. The candidate, however, knows this is wrong and would get them out.",
        "selection": [
            1,
            2
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/DwV8o9DzIRc\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 374 30448",
        "url": "https://www.youtube.com/watch?v=ELVinOCD_10",
        "A": "The candidate is shown delivering the State of the Union address at the beginning of the term as president. The narrator echoes the candidate's resolve and accomplishments, as the ad displays a message urging voters to support the candidate in the election. ",
        "B": "The current president gives the State of the Union, emphasizing the desire in America for strength, freedom, and peace entering into the new decade. He claims that this America can be achieved working together. The advertisement explains the presidential candidate to be a man of resolve and achievement. ",
        "selection": [
            1,
            3
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/ELVinOCD_10\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 475 96893",
        "url": "https://www.youtube.com/watch?v=EvX3yaiVKeg",
        "A": "In a scolding tone, a female narrator chastises the opposing candidate, depicted as an incompetent child by voice, for budget discrepancies. The opposing candidate is forced to admit to overspending by neglecting permanent tax costs, implementing a prescription drug plan, and proposing Social Security privatization. ",
        "B": "The ad discusses the opposing candidate's budgets and how the numbers do not add up and the candidate has overspent by 6 trillion dollars. These hidden costs are examined and the ad criticizes the candidate, breaking apart the math as a teacher as if the candidate were a child.",
        "selection": [
            1,
            2
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/EvX3yaiVKeg\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1197 52122",
        "url": "https://www.youtube.com/watch?v=F7Y5Aliw5fg",
        "A": "The ad starts out with a scene regarding the candidate's past in the navy and goes on to emphasize his belief in preserving peace through a strong national defense. The candidate's record in diplomacy and as Director of the C.I.A. is used to portray him as a president who is already ready for the position.",
        "B": "Film showing soldiers rescuing a downed American flyer in Japanese-patrolled waters during World War II. That flyer was the presidential candidate. His goal is to maintain peace with strong national defense. His experience in diplomacy and as director of the CIA make the candidate an excellent leader.",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/F7Y5Aliw5fg\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1154 50000",
        "url": "https://www.youtube.com/watch?v=FpO-oYwqlZ0",
        "A": "The ad accuses the opposing candidate of dodging debates and attacking the candidate. The ad states facts about the candidate such as job growth within Arkansas, the low tax burden, and the large move from welfare to work. The ad strives to show how the candidate has a great record.",
        "B": "The advertisement explains that the current president, and opposing candidate, has been attacking the candidate and avoiding debates. When focusing on the record, however, it appears that the state the candidate governs is doing better than the country in terms of economic performance. The candidate challenges the president on this.",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/FpO-oYwqlZ0\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1182 51824",
        "url": "https://www.youtube.com/watch?v=Hq4CgJbP0ZU",
        "A": "The candidate is said to take advantage of every possibility to encourage exchanges between free men; these exchanges can lead to genuine peace which is extremely important, especially the Middle East. The president is labeled as more of a peacemaker than recent presidents and is said to understand the hope and promise America holds.",
        "B": "The advertisement describes Abraham Lincoln's idea that America was the world's last hope, and this has guided the course of the candidate's presidency. He has served as an international figure on peace and has put peace above any personal gains. With this goal in mind, the candidate seeks re-election.",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/Hq4CgJbP0ZU\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 359 9968",
        "url": "https://www.youtube.com/watch?v=IQnLNhavI_4",
        "A": "The candidate gives a speech to an audience. He emphasizes, regardless of personal politics, that what he has to say is important. He explains, in the context of social reform, that if a peaceful change is impossible, then a violent change is inevitable. This advises the importance of peaceful change. ",
        "B": "The ad features a candidate giving a speech urging the people to take one thing away from this regardless of their party affiliation, etc.: if a peaceful change of conditions plaguing current society is impossible, then a violent change is inevitable. ",
        "selection": [
            3,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/IQnLNhavI_4\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 500 94375",
        "url": "https://www.youtube.com/watch?v=M4cOa7ZAEz4",
        "A": "The candidate stands next to stacks of papers indicated to be Congress's federal budget. He explains that the president cannot choose which parts of the budget to support, but he could with a line-item veto that would allow for lower federal debt. This is why the candidate should be president.",
        "B": "The candidate speaks on the federal budget given to the president by Congress and how the president must buy it all or say no to everything. The candidate emphasizes that if the president could have item vetos, the federal deficit could be cut, and urges for action which he can do.",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/M4cOa7ZAEz4\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 500 94360",
        "url": "https://www.youtube.com/watch?v=NmM0nZW5Rk0",
        "A": "The advertisement describes the candidate's various experiences: outing corruption as a news editor, working for civil rights as a state legislator and lieutenant governor, and serving as a congressman and senator. His story is one of hope and being able to believe again.",
        "B": "The ad discusses the candidate's past as an editor who exposed corruption, an Illinois legislator and lieutenant governor who risked his career over civil rights, and then as a congressman and senator battling for arms control, education, and jobs. The ad portrays the candidate as someone of hope, courage, and principle.",
        "selection": [
            1,
            3
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/NmM0nZW5Rk0\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1051 43133",
        "url": "https://www.youtube.com/watch?v=OYEevxgHATM",
        "A": "Voters in New Hampshire describe their concerns with the current president. They explain that the president has failed in his promises that he made to them in economic policy especially, and they can no longer trust him. The viewers are urged to send a message by voting for the candidate.",
        "B": "Individuals around the country speak on an opposing candidate's inability to follow through, labeling it as betrayal. They speak on the different issues he went against his word on and how they affected them personally. The candidate urges the people to vote for him and send a message to the opposing candidate.",
        "selection": [
            1,
            2
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/OYEevxgHATM\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1051 43126",
        "url": "https://www.youtube.com/watch?v=PG4CD0rZQB0",
        "A": "With patriotic music playing, the candidate explains the need for a president who can stand up to Congress. As president, he would demand a hiring and spending freeze, enact a moratorium on new federal regulations, limit spending, and place term limits on Congress. He calls for a \"new American revolution.\"",
        "B": "The candidate urges the people to vote for someone who will stand up against Congress. The candidate says he will do exactly this and work to stop wasteful spending, put term limits on every member of Congress, and demand a hiring and spending freeze. As president, he will create a new American revolution.",
        "selection": [
            1,
            2
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/PG4CD0rZQB0\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1009 40497",
        "url": "https://www.youtube.com/watch?v=SjcOHwt24Uo",
        "A": "A senator speaks on his views of the candidate. He says that the candidate has necessary and new answers. The senator tells a story of a long-term Democrat being frustrated with the Vietnam war and inadequate paychecks and pensions. This Democrat chooses to vote for the Republican candidate.",
        "B": "A Senator of Tennessee speaks about the candidate and how he will get votes potentially from unexpected groups: those who never voted Republican but now have children or family in Vietnam, those who are concerned about paying their bills, etc. More people will vote for the candidate when considering their children's futures.",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/SjcOHwt24Uo\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1009 40490",
        "url": "https://www.youtube.com/watch?v=Umjh7iaZhM4",
        "A": "A senator spokesperson converses about the candidate's resolution, knowledge, and purpose that is crucial for addressing foreign policy issues. Emphasizing the candidate's ability to maintain military strength as key to negotiating peace, especially in the Middle-East, the senator expresses approval for the candidate's foreign policy management capability. ",
        "B": "A senator of New York discusses the candidate and how he has the strengths to deal with the foreign policy of the country at that time, such as resolution, purpose, and knowledgeability. He says electing this candidate will help work on correctly negotiating peace and running foreign policy.",
        "selection": [
            2,
            3
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/Umjh7iaZhM4\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 387 30797",
        "url": "https://www.youtube.com/watch?v=UsX7c8C9wQc",
        "A": "The candidate introduces himself while photographing various farm scenes, emphasizing Iowa's beauty as a reflection of America's strength. The candidate seeks out leadership that strengthens farming education and export programs and urges viewers in agreement to take part in the caucus in order to restore America's special strengths. ",
        "B": "The candidate appeals to Iowa farmers and the state's strengths in America. He supports policies that would allow for greater food production, trade offensives, and farm export promotion. The Iowa caucuses are emphasized as special, and the candidate urges Iowans to take part to help make America special again.",
        "selection": [
            3,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/UsX7c8C9wQc\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1232 57581",
        "url": "https://www.youtube.com/watch?v=VyHaEZ9kex0",
        "A": "The ad critiques an opponent for giving weekend passes to murderers not eligible for parole, vetoing mandatory sentences for drug dealers, and opposing the death penalty. The ad uses all of these to ask the voters who they can really count on to be tough.",
        "B": "The advertisement compares the opposing candidate's policies on crime to the candidate. The opponent is portrayed as weak on issues of parole, mandatory sentencing, and capital punishment for police officer murders. The candidate, on the other hand, can be counted on for being tough on crime with his experienced leadership.",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/VyHaEZ9kex0\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1064 43975",
        "url": "https://www.youtube.com/watch?v=XvgSx9shFtQ",
        "A": "Focusing on a photo of the candidate, the narrator emphasizes the candidate's importance to the United States. A young first-time voter highlights their generation's desire for peace and progress without compromising national security. They express their intention to vote for the candidate based on their perceived commitment to safeguarding voter interests. ",
        "B": "The advertisement shows a student and first-time voter explaining his voting plans. His generation wants peace and progress without sacrificing national security, and this candidate is the one who can provide that. He will vote for the candidate because he thinks he will best protect his interests.",
        "selection": [
            1,
            3
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/XvgSx9shFtQ\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 316 8988",
        "url": "https://www.youtube.com/watch?v=_PwIPr40mjE",
        "A": "A dairy farmer describes his concerns with inferior-quality dairy imports coming to the United States and necessary legislation to prevent this. Such legislation would protect dairy farmers and American consumers. The advertisement ends with the assurance that the candidate is the one to address these concerns.",
        "B": "A Wisconsin dairy farmer advocates for legislation to uphold high-quality dairy standards from a farm, expressing concern over inferior imports negatively impacting domestic farmers and consumers. The narrator asserts that the endorsed candidate could bet the solution to address and solve these concerns. ",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/_PwIPr40mjE\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1033 42062",
        "url": "https://www.youtube.com/watch?v=ahVNtk4YoXk",
        "A": "The ad features a candidate advocating for the US to win the economic competition and peace by becoming a military, economic, and export superpower. The candidate emphasizes that there are two versions to do so: the opposing candidate wants to look inward while the candidate will look forward by making immense changes.",
        "B": "The candidate describes a world in transition, and thus, America must also transition. America must be a superpower in the economy, military, and exports. While being incredibly patriotic in his speech, he outlines needed policies in economic, legal, health, workforce, and education fields to move America into the new century.",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/ahVNtk4YoXk\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1012 40696",
        "url": "https://www.youtube.com/watch?v=bCD4myTnyVI",
        "A": "The opposing candidate from the same party is wrong about the candidate. The candidate plans to stimulate the economy through tax cuts for the middle class with tax increases for the rich, while the opposing candidate disapproves of such a policy. The opponent represents the old party style.",
        "B": "The ad criticizes the opposing candidate for having the wrong facts about the candidate he is attacking. The ad clears this up by stating that the candidate is not for tax breaks for the rich but the exact opposite. The opposing candidate is criticized for being an old-style negative democrat. ",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/bCD4myTnyVI\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1289 59973",
        "url": "https://www.youtube.com/watch?v=bIfuzyuOq-Q",
        "A": "The ad features a speech by the candidate about America in 1980 and how by working together, they brought interest rates down, inflation down, and got America working again. The ad strives to show how the candidate's leadership is working and will continue to. ",
        "B": "The advertisement displays images of a stagnant economy with rising inflation as how the nation was in 1980. Today, however, interest rates and inflation are down, Americans are working again, and the nation will carry on. The current president (and candidate) describes this as a goal for re-election.",
        "selection": [
            1,
            2
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/bIfuzyuOq-Q\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 601 114472",
        "url": "https://www.youtube.com/watch?v=boT4guTZnqs",
        "A": "The candidate discusses the difference between being president and running for president. He emphasizes how political slogans end up being worth nothing after elections. He highlights his campaign message and the importance of action with the plan he has to get the country out of Iraq. ",
        "B": "The candidate explains the roles of the president and candidates' promises versus actions. The candidate plans to act and has shown so through his plans to end the war in Iraq. This plan has bipartisan support. He has already shown political courage to act and would do more as president.",
        "selection": [
            1,
            2
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/boT4guTZnqs\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1190 35259",
        "url": "https://www.youtube.com/watch?v=c7o-DT0FvXU",
        "A": "The candidate discusses how he achieved every goal that he set when campaigning in 1980; goals such as lowering inflation, crime, and taxes have been achieved. He urges voters to continue to build lasting peace and security as well as improving education, the job market, and crime with him in office as their leader. ",
        "B": "The current president describes his campaign promises and how he has fulfilled these in office. In seeking re-election, the candidate describes work that still needs to be done for further improvements. America doesn't stop trying to be better and always tries to be best. The candidate's leadership is working.",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/c7o-DT0FvXU\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 380 30565",
        "url": "https://www.youtube.com/watch?v=cZ7aNeSgorQ",
        "A": "The ad critiques the opposing candidate and questions whether someone who has sent troops all around the world and even destroyers should be the person we have in office. The ad shows that the candidate is a man of peace and is respected around the world, urging them to vote right.",
        "B": "The advertisement describes the opposing candidate as a hostile man who would send military to foreign nations over minor disputes, and questions whether this is the president Americans want. In comparison, the candidate is known around the world for his role in peace. The choice is up to voters.",
        "selection": [
            3,
            2
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/cZ7aNeSgorQ\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 475 96879",
        "url": "https://www.youtube.com/watch?v=cyjIhJoYo6I",
        "A": "With tense music, the ad depicts panels of busy men alongside warnings of a special interest feeding frenzy, giving billion to drug companies, or ceding wilderness to big oil. The candidate is shown advocating and leading against these interests, endorsed with a slogan highlighting his courage to do what is right. ",
        "B": "The advertisement emphasizes the candidate's role in preventing special interest drug companies and oil drilling operations in America. As president, the candidate would continue these policies to lower prescription drug prices, take on the insurance industry, and make America more energy independent. Viewers are urged to support him.",
        "selection": [
            1,
            3
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/cyjIhJoYo6I\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1232 57589",
        "url": "https://www.youtube.com/watch?v=d7NFB4jz9v8",
        "A": "The advertisement appeals to Hispanic voters who may be affected by the presidential election. A Hispanic woman explains that with diverse needs, it is important that a president listens to Hispanic Americans. The candidate stands next to this woman and her kids, revealing that the candidate is her father-in-law.",
        "B": "The ad features a Hispanic woman emphasizing why this election is important. She discusses how Hispanic people now have a real chance to be heard and have a person in office who cares about their needs and traditions. The woman is the candidate's daughter-in-law and helps vouch for his credentials.",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/d7NFB4jz9v8\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1085 45276",
        "url": "https://www.youtube.com/watch?v=fl9u2cDabJs",
        "A": "The candidate discusses the tax cut and how it cannot do anything for the people. He speaks on the current state of the economy and how the country is already in debt and will harm future generations by going more into debt to offer these tax cuts. He advocates \"straight talk for a change\". ",
        "B": "The advertisement describes opposing candidates' policy of extreme tax cuts while comparing it with one that would create jobs. The candidate explains that this is poor policymaking because of the country's debt and the necessary borrowing this would require. The candidate wants to focus on the presidency instead of giveaways.",
        "selection": [
            1,
            2
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/fl9u2cDabJs\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 603 114555",
        "url": "https://www.youtube.com/watch?v=hWbpWxG-AkM",
        "A": "The candidate is portrayed as a rare conservative in a liberal state. The candidate highlights his accomplishments, such as tax cuts and immigration law enforcement, and values, such as traditional marriage and pro-life stances. The candidate affirms the importance of standing up for conservative values. ",
        "B": "The advertisement describes the candidate's strength in working in the most liberal state in the country yet standing with his party's values. He has shown his ability to stand for a strong military, a strong economy, and traditional families. His values mark his ability to lead the country as president.",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/hWbpWxG-AkM\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 365 2168",
        "url": "https://www.youtube.com/watch?v=iRYYBFRtfXI",
        "A": "The advertisement appeals to working-class Americans and the candidate's shared ability to conquer challenges. To this audience, he emphasizes the need for less government interference. The influence of Indiana in the presidential race is explained, and the candidate wants to earn support from the state to become president.",
        "B": "The ad features the candidate speaking to workers and other individuals. It highlights that this could be the year where Indiana plays a decisive role in the election and encourages voters who agree with the candidate that we must do better than we have done to vote for him.",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/iRYYBFRtfXI\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 329 1698",
        "url": "https://www.youtube.com/watch?v=lrLA6TVqap0",
        "A": "The ad emphasizes the importance of supporting the candidate through time or financial contributions if you believe in the candidate's necessity. Urging for support, the message highlights how donations can contribute to the candidate's presidential bid. Contact information is displayed for those willing to donate or volunteer. ",
        "B": "The advertisement describes other Americans who support the candidate. It explains that if viewers also support the candidate, their financial contributions are requested. Such action is necessary in order to ensure the success of the candidate. The advertisement ends listing various paths the viewer can take to make a donation. ",
        "selection": [
            1,
            2
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/lrLA6TVqap0\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 500 94349",
        "url": "https://www.youtube.com/watch?v=mA-47rYYSLs",
        "A": "The ad features an intelligent and successful student who cannot afford to go to college due to tuition and how there are many families in the same position. The candidate's college opportunity plan supports the idea that if a student has the grades for college, America should find a way to send them.",
        "B": "The advertisement follows Jimmy, a young American who is successful in school but does not have the financial ability or scholarships to be able to attend college. The candidate has a policy goal that Jimmy and other students like him would be able to attend college in America.",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/mA-47rYYSLs\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1084 45260",
        "url": "https://www.youtube.com/watch?v=nLPvTS_iMTI",
        "A": "The candidate's strong leadership is discussed, his vision to strengthen the economy and make America more competitive, to change welfare, reform the court and end fraudulent lawsuits, and reform education. The candidate emphasizes the future of America and the experience he has to shape it.",
        "B": "The current president is shown in the Oval Office, with claims of being the strongest leader America has seen. His new goals include strengthening the economy and making America more competitive. He also plans welfare, court, and education reform. The advertisement emphasizes his experience with America in securing its future.",
        "selection": [
            1,
            2
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/nLPvTS_iMTI\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1186 51958",
        "url": "https://www.youtube.com/watch?v=oI-cSCG2YCU",
        "A": "The advertisement explains that the current president entered office under difficult conditions and has since turned the economy around for America. He has made hard decisions and rejected excessive spending without making empty promises. He plans to maintain economic recovery without inflation, and thus should be re-elected.",
        "B": "The ad exemplifies how the candidate is leading the country out of its worst recession. The ad emphasizes that he has made hard decisions, has dared to say no to spending, and as a result, has led inflation to be cut in half, helped 2 million Americans go to work, and helped America recover. ",
        "selection": [
            1,
            2
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/oI-cSCG2YCU\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 365 2164",
        "url": "https://www.youtube.com/watch?v=r2LsLW9LgdM",
        "A": "The ad discusses the man that Kennedy sent around the world to speak for America in 1962 and the hope and concern he brought with him. The ad highlights his achievements and tells voters that Indiana can help pick the next president and ensure this type of action continues. ",
        "B": "In an Indiana targeted advertisement, the candidate is portrayed as globally experienced, having represented America around the world. The candidate is portrayed with a cheering crowd and friendly demeaner through video evidence. The footage also showcases his accomplishments around the world, establishing friendly relations and dealing with critics. ",
        "selection": [
            2,
            3
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/r2LsLW9LgdM\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 334 15180",
        "url": "https://www.youtube.com/watch?v=tYo8pu-rOLU",
        "A": "It is asked which candidate passed legislation to keep oil prices down. It lists every candidate but finds just one. Another question is asked about environmentalism; again, the same candidate wins here. The last question deals with exportation of American jobs, and again, the candidate comes out on top.",
        "B": "The ad asks viewers a series of questions regarding whose bill got passed to put a lid on oil prices, who won a highest environmental award, and who insists that we stop exporting our jobs to Russia. They all are the same candidate that the ad strives to encourage viewers to vote for.",
        "selection": [
            3,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/tYo8pu-rOLU\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1332 64012",
        "url": "https://www.youtube.com/watch?v=xSNqTxLyBAI",
        "A": "The candidate discusses how he handles matters in the South. Rather than using scapegoats, he emphasizes that they work hard and improve schools, workforce, factories, and more to outcompete the people of any nation. He urges voters to vote for him and help put America back on top.",
        "B": "The candidate explains that blaming everything on foreign countries is not how things are done in the South, appealing to that region's voters. His goals for America include improving schools, retraining the workforce, and modernizing factories. This would allow America to think, produce, and compete more than any other nation.",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/xSNqTxLyBAI\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 140 10498",
        "url": "https://www.youtube.com/watch?v=0uZ-oQhnJ3k",
        "A": "The narrator compares the candidate to esteemed past Democrats renowned for their approachability and relatable personas, equalities the candidate also embodies. The ad depicts scenes of the candidate interacting warmly with children, shaking hands, and smiling to convey a friendly and approachable image. ",
        "B": "The advertisement details a type of person that Americans want to reach out and \"touch.\" Among this group is the candidate running for president. Pictured in the video is said candidate physically embracing audiences. The candidate has worked hard for the common man, and this is what Americans want.",
        "selection": [
            1,
            2
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/0uZ-oQhnJ3k\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 142 5896 wm",
        "url": "https://www.youtube.com/watch?v=1ooE3txlwwY",
        "A": "The advertisement details that the candidate is the only one in the polls who can beat the opposing party's candidate. No other member in the candidate's party has been able to beat the other party, and thus, viewers are urged to vote for the candidate.",
        "B": "The narrator reads from a newspaper headline declaring the candidates victory over an opposing party's candidate in a national Gallup poll. Emphasizing this achievement, the campaign ad highlights that only this candidate will be able to secure victory against the opposing party. ",
        "selection": [
            1,
            3
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/1ooE3txlwwY\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 36 1008",
        "url": "https://www.youtube.com/watch?v=3cdIVdCqD4g",
        "A": "The advertisement begins with a workplace closing. The opposing candidate voted against a law that would require 60 days' notice before shutting down a plant. The candidate and his running mate, however, support community and workers' rights, having passed a law that requires notification of plant closings.",
        "B": "Accompanied by ominous music, an image of a locked company gate is used while discussing job loss and insecurity, highlighting the importance of a law mandating prior notification of company closures. While the opposing candidate is cited to have opposed this legislation, the endorsed candidates have championed such legislation. ",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/3cdIVdCqD4g\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 214 12350",
        "url": "https://www.youtube.com/watch?v=62zHpgnq3bE",
        "A": "The ad states that presidential candidate Carter used to be for laws that protected the rights of workers when he was governor.  But now as a presidential candidate, with the support of big labor unions, he has become committed to signing a bill that would remove these right-to-work laws.",
        "B": "An image presented shows the opposing candidate initially on one side of the screen, supporting Right to Work laws, then flips to the other side of the screen to show him now opposing them. This inconsistency and lack of support is cited as a reason to support the endorsed candidate. ",
        "selection": [
            3,
            2
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/62zHpgnq3bE\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 24 373",
        "url": "https://www.youtube.com/watch?v=6Ucmx9-HDSc",
        "A": "The candidate discusses various social issues that have led to a decline in American sentiment held by young Americans. He emphasizes that while America has its problems, it is the nation with the most opportunity, freedom, and promise for young Americans compared to any other country or government system.",
        "B": "The candidate discusses how young Americans have lost faith in America. He emphasizes that they are against everything. He shifts to claim that he has seen other systems and nations and can confidently say that the United States is the best and has the most opportunities for young people.",
        "selection": [
            1,
            3
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/6Ucmx9-HDSc\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 1 18",
        "url": "https://www.youtube.com/watch?v=6zJB-6_RA8c",
        "A": "As part of the candidate's campaign series answering questions from Americans, a father expressed concern about assuming debt to support his family. The candidate emphasizes that this experience is shared amongst Americans that spent more than they earned, on average. Warning of serious consequences, the candidate advocates for change. ",
        "B": "A father stands with his son and explains personal issues with debt and supporting his family. The candidate recognizes these grievances with the current political situation. He explains that something needs to change, indicating that he is the candidate who could make that change.",
        "selection": [
            3,
            2
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/6zJB-6_RA8c\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 2 00023",
        "url": "https://www.youtube.com/watch?v=8qzTT9vUJNY",
        "A": "The ad features an individual who endorses the candidate and emphasizes that this is no time for an amateur. The ad highlights that now than ever, the White House needs someone who knows civilian government. The ad urges voters to switch to the candidate.",
        "B": "A man describes his thoughts on the upcoming election. He has come to the conclusion to vote for the candidate. Because an amateur would hurt America, and the candidate is an expert in civilian government, the man is voting for him. The man even switched his vote for the candidate.",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/8qzTT9vUJNY\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 9 147",
        "url": "https://www.youtube.com/watch?v=90dTB8FK-70",
        "A": "The advertisement begins with a negative remark about the opposing candidate's focus on speech-making in the election. The sitting president recommends using the virtues of character, ability, responsibility, and experience to evaluate the candidates. He endorses the candidate and his running mate whom he trusts with these values.",
        "B": "The candidate is endorsed by a previous president who discusses four key qualities that are necessary for an individual to be a proper candidate for president: character, ability, responsibility, and experience. The candidate is said to possess these abilities and is thus, deemed the proper individual to vote for. ",
        "selection": [
            3,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/90dTB8FK-70\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 32 460",
        "url": "https://www.youtube.com/watch?v=9va7OvcEjEU",
        "A": "The advertisement depicts an income tax form. The candidate explains that there are many tax loopholes that are not accessible to typical working Americans. However, the wealthiest Americans use these loopholes regularly. He recognizes that these are the Americans who need breaks the least, whereas ordinary Americans need them most.",
        "B": "A tax form is displayed highlighting taxes on manual labor. The ad critiques hidden loopholes benefiting select groups while working Americans are left disadvantaged. The candidate introduces themselves, alleging that loopholes disproportional favor the affluent, leaving hardworking Americans cheated. ",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/9va7OvcEjEU\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 8 130",
        "url": "https://www.youtube.com/watch?v=CHdozzBzYU0",
        "A": "When confronted with an accusation regarding economic stagnation, the candidate responds by pointing to all time increased earning for many Americans. Referencing a past administration marred by inflation, the candidate highlights their own administration's achievement of real wage increases, asserting this financial progress is crucial for peace. ",
        "B": "The candidate discusses how Americans are now earning more than ever compared to the previous administration. He calls this honest economic growth and promotes the idea of growing strong so the country can be at peace, which can only be done with him in office.",
        "selection": [
            1,
            2
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/CHdozzBzYU0\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 137 10472",
        "url": "https://www.youtube.com/watch?v=F5fYkqCEsRQ",
        "A": "A spokesperson for the candidate admits that despite considerable support in a previous election, the candidate's campaign lagged begin financially compared to the well-funding winning opponent. They urge supporters to contribute to the candidate's campaign, ensuring a fair chance at victory. ",
        "B": "The advertisement includes a man explaining that if some of the people who voted for the candidate in the previous election had donated to the campaign, the opposing candidate may not have been elected. By donating now, though, the campaign could offset the big donors the opposing candidate receives.",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/F5fYkqCEsRQ\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 21 300",
        "url": "https://www.youtube.com/watch?v=Fa3MJIMXat8",
        "A": "Then-Governor Ronald Reagan explains his support for the candidate's presidential ticket. He explains that voting for a third party would be wasting a vote; therefore, he calls for voters to unite under the Republican Party and their support of the candidate. That party allows freedom, action, and change.",
        "B": "Governor Reagan endorses the candidate, asserting that Republicans stand behind him. Concerns about wasted votes in crossing party lines are raises and it is suggested that those seeking genuine change should opt for the Republican candidate, instead of wasting votes on third parties, to ensure progress towards individual freedom and less government dominance. ",
        "selection": [
            3,
            2
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/Fa3MJIMXat8\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 23 348",
        "url": "https://www.youtube.com/watch?v=IYHRMlrEfqU",
        "A": "The ad discusses how the opposing candidate wishes to cut many parts of the US defense and military. The candidate being supported does not endorse this and believes in a strong America and peace, which can come from strength. This means investing in the nation's defense program and security, which he firmly believes. ",
        "B": "Visual imagery of toy army men and models is used to depict the opposing candidates plans to reduce military funding, which, a senator claims would undermine national security. The candidate, portrayed alongside military officers as \"Hail to the Chief\" plays, emphasizes a commitment to negotiating peace from a position of strength. ",
        "selection": [
            3,
            2
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/IYHRMlrEfqU\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 43 2114",
        "url": "https://www.youtube.com/watch?v=KylHex5XlW0",
        "A": "The advertisement includes elderly women describing their experiences with food stamps and assistance programs. They explain that their benefits had been cut, leaving it difficult for them to manage on the little they now receive. The advertisement ends with assurance that the candidate would be able to solve this issue.",
        "B": "Elderly women at a senior home express hardship, mentioning reduced food stamp benefits and insufficient support to meet their needs. They emphasize the impossibility of enduring further cuts to their benefits. Implicitly, the candidate is portrayed as empathetic to these struggles, urging voter support. ",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/KylHex5XlW0\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 58 2460",
        "url": "https://www.youtube.com/watch?v=PIY0WGfIdAI",
        "A": "The candidate describes concern with the current administration's economic policies. In his role as governor, however, he cut excessive government spending and turned a large deficit into a surplus. As president, he would do the same for America, and thus, it is the ideal time for his presidency.",
        "B": "The candidate asserts that Americans are deeply concerned over the economic shortcomings of the past administration. Leveraging his past tenure as a governor, the candidate highlights his success in collaboration between officials and private citizens to reduce government deficits, even achieving surplus and asserts that he could do it again. ",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/PIY0WGfIdAI\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 156 1228 wm",
        "url": "https://www.youtube.com/watch?v=QoT5Ap10p40",
        "A": "The ad discusses the difference between the republican candidates and how only one grew up during the Depression, has had wounds from WWII, been a republican leader in the Senate, cut taxes, and reduced wasteful spending. The emphasizes that the candidate is one of the people and understands their needs.",
        "B": "The advertisement outlines the differences between candidates from the same party. The candidate comes from a rural working-class background and has served in the military. He is a leader in the Senate to reduce government spending and taxes. As president and \"one of us,\" he would meet Americans' needs.",
        "selection": [
            3,
            2
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/QoT5Ap10p40\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 218 1380",
        "url": "https://www.youtube.com/watch?v=S-jvmoro52E",
        "A": "The opposing candidate is depicted as a figure on a wind vane, pointing in various directions, symbolizing inconsistency and unreliability. The narrator criticizes the opposing candidates lack of dependability and trustworthiness, recommending voting for the endorsed candidate instead. ",
        "B": "The advertisement displays a wind vane with the head of the opposing candidate atop it. It describes the opponent taking conflicting stances with regard to open housing legislation. It is implied that this shift in opinion is unpredictable and unwanted. The viewers are instead encouraged to vote for the candidate.",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/S-jvmoro52E\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 48 2255",
        "url": "https://www.youtube.com/watch?v=S5UWIkD3lQg",
        "A": "The ad discusses property taxes and the burden that the opposing candidate will place on families if this is done. It discusses the increase and its repercussions. The ad urges the public to vote for the supporting candidate on election day.",
        "B": "Accompanied by an image of a house, the narrator critiques the opposing candidates' economic plans and suggests that they will shift costs onto local governments through property taxes. A potential tax increase outcome for families of four in Ohio is displayed and voter are urged to vote for the candidate. ",
        "selection": [
            3,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/S5UWIkD3lQg\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 214 12337",
        "url": "https://www.youtube.com/watch?v=UXKzUIBwoGg",
        "A": "The advertisement begins by explaining that the current president can easily beat his opponent. Americans are interviewed on their opinions of the opposing candidate running for president. They detail not remembering what he accomplished in serving as a state governor, or that he did nothing in his position.",
        "B": "A friendly image depicts the candidate extending a hand. Spokespersons from major cities then assert that the candidate's background positions them to defeat the opposing candidate, citing dissatisfaction with the previous administration's inaction. Former constituents from the opposing candidate's governorship express uncertainty about his accomplishments. ",
        "selection": [
            1,
            3
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/UXKzUIBwoGg\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 195 12114",
        "url": "https://www.youtube.com/watch?v=aZB0aWSQCdE",
        "A": "The candidate highlights that, at the time, 200 corporations possessed two-thirds of all manufacturing assets, fostering monopolies, job exports, and price fixing. Citing the need for fair competition, the candidate pledged to fight against fixed pricing, advocating for fairer economic practices. ",
        "B": "The candidate discusses corporations and the monopolies they have which leads to them fixing prices. The candidate then argues that competition is needed to keep these corporations from fixing prices and he will be the one to do that if elected.",
        "selection": [
            1,
            2
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/aZB0aWSQCdE\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 5 00083",
        "url": "https://www.youtube.com/watch?v=basW7Xngx6A",
        "A": "A children\u2019s doctor endorses the candidate due to the candidate\u2019s platform for federal aid to education. The doctor discusses the candidate\u2019s time as a senator, working for aid, scholarship, and more while the opposing candidate has voted against such measures; thus, the doctor has faith in the candidate\u2019s ability.",
        "B": "A physician introduces his concerns with children\u2019s health, security, education, and futures. He speaks in support of the candidate, mainly due to his public education policies. The physician details the negative education voting record of the opposition candidate. He broadens his full support to the candidate\u2019s stance on other policies.",
        "selection": [
            3,
            2
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/basW7Xngx6A\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 293 14367",
        "url": "https://www.youtube.com/watch?v=bdC7-aTJ0Xk",
        "A": "The ad viewers the candidate who walks through the promises he had made to the public when he was campaigning and how his administration achieved all of those goals. The candidate then urges that more can be done and will be done and America can be bettered by reelecting him. ",
        "B": "The sitting president describes campaign promises made in the past and his success in fulfilling these. He goes on to explain the work left to be done, and how it will be accomplished through his re-election. His plans seek to improve quality of life for Americans and secure America's future. ",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/bdC7-aTJ0Xk\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 10 180",
        "url": "https://www.youtube.com/watch?v=eCLDRDUTdpc",
        "A": "The scene depicts a little girl plucking and counting flower petals. It closes in on her face with a man counting down in the background. The countdown leads to explosions, emphasizing the threat of nuclear warfare, especially for children. The candidate elaborates on this fear, explaining peace is necessary instead.",
        "B": "The ad endorses a candidate by portraying how he is willing to do what it takes to make the world a proper place for us all to live in. It emphasizes how the stakes are too high not to vote and how this candidate plays a crucial role. ",
        "selection": [
            1,
            3
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/eCLDRDUTdpc\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 147 10600 wm",
        "url": "https://www.youtube.com/watch?v=gZZ-5MODFj0",
        "A": "The candidate shows voting records including the votes of other presidential candidates. The opposing candidates had voted to allow racialized busing of students. Instead, Floridians want free choice and neighborhood schools, which the candidates have since changed their minds and support. The candidate already supports this and should be elected.",
        "B": "The candidate displays congressional records and notes that Florida senators have opposed freedom of choice and neighborhood schools in favor of busing for racial balance. Though some have changed their stance due to the public pressure, the candidate urges supporters to contribute to their campaign to further promote these issues. ",
        "selection": [
            3,
            2
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/gZZ-5MODFj0\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 54 387",
        "url": "https://www.youtube.com/watch?v=jV1xNlS-D9s",
        "A": "The candidate addresses a crowd, with a women in focus, emphasizing his commitment to promoting women's rights during a critical period. The candidate highlights his support for extending legislation on women's rights, positioning himself as the sole advocate among candidates. His slogan reinforces his identity as a unique candidate.  ",
        "B": "The candidate discusses how women are an important part of his constituencies. Further, he emphasizes women's right and the ERA, highlighting that there is only one administration that has worked to get the extension needed to secure ratification. The ad urges viewers to consider the difference between this campaign and the others. ",
        "selection": [
            1,
            2
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/jV1xNlS-D9s\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 2 30",
        "url": "https://www.youtube.com/watch?v=nCoTHLjPdl8",
        "A": "In a rendition sung to the tune of \"America the Beautiful,\" a formally dressed woman encourages voters to support the candidate. The song lauds the candidate's humble origins and achievements, and highlights their commitment to peace amidst other candidates' focus on war. ",
        "B": "A woman sings throughout the advertisement in support of a candidate. She connects him to Abraham Lincoln in being from Illinois and in his leadership abilities. Comparing the candidate to those with military experience, she notes that others are focused on war, while the candidate has a peace mindset.",
        "selection": [
            1,
            2
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/nCoTHLjPdl8\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 143 5949 wm",
        "url": "https://www.youtube.com/watch?v=qMyCSULFmqE",
        "A": "The ad features an individual asking what the candidate would do to stop the rising high cost of living. The candidate states he would cut down government spending, keep the size of government down, and ensure that the free competitive enterprise system of America can flourish. ",
        "B": "A man opens with a question for the candidate regarding the worth of a dollar and the cost of living. The candidate responds with ways he would correct mistakes of the current administration. He plans to cut government spending and the size of government, while creating jobs with real wages.",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/qMyCSULFmqE\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 17 257",
        "url": "https://www.youtube.com/watch?v=t3VInjYfvks",
        "A": "The candidate discusses how he would differ as president versus his current position as vice president. In addition, he highlights that he would be a leader and take charge of the presidency in his own manner, an approach different than the one he has taken as vice president.",
        "B": "During a National Press Club event in Washington, the candidate emphasizes his role as an individual thinker and collaborative team member to his administration regardless of the fact that he assumes a vice-presidential role. ",
        "selection": [
            3,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/t3VInjYfvks\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 13 212",
        "url": "https://www.youtube.com/watch?v=tiou_m2cLmg",
        "A": "The question of whether the candidate supports the United Nations is asked. The candidate supports the United Nations but wants to see it come closer to its original purpose, urging that the United States bring a clear foreign policy and demand that all members do its part.",
        "B": "A man opens asking a question about the candidate's thoughts on the United Nations. The candidate explains his support for the United Nations, which is contingent upon it coming closer to its original purpose. He argues that improvements of equal payments, unanimous agreement, and anti-communism should be put in place.",
        "selection": [
            3,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/tiou_m2cLmg\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 10 181",
        "url": "https://www.youtube.com/watch?v=txK2TTwiWEc",
        "A": "The ad discusses atomic bombs and the threat of them on children. Further, it discusses how the opposing candidate if elected, will threaten the Nuclear Test Ban treaty and start testing bombs again, raising radiation and harming the children of the country once again.",
        "B": "The advertisement depicts a young girl licking an ice cream cone. The audio contrasts the type of America this child would face under the opposing candidate; the child's experience would likely involve harmful testing of atomic weaponry. Under the candidate's administration, however, the child could lead a normal life.",
        "selection": [
            3,
            2
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/txK2TTwiWEc\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 8 138",
        "url": "https://www.youtube.com/watch?v=uUMbaenVjsI",
        "A": "The candidate addresses the audience directly, emphasizing the importance of strength, firmness, courage, and peace for the future. The candidate implies that voting for him will allow for the safety and peace of future generations of children. ",
        "B": "The candidate speaks directly to the audience, expressing his concern with being strong and firm with goals of peace in a divided environment. He stresses the importance of long-lasting peace for the country. The advertisement ends with a recommendation to vote for the candidate and his running mate.",
        "selection": [
            1,
            3
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/uUMbaenVjsI\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 150 10702",
        "url": "https://www.youtube.com/watch?v=vSwQjWJIljc",
        "A": "The ad discusses the opposing candidate's defense plan which consists of cutting a portion of the marines, air force, navy, interceptor planes, navy fleet, and carriers. The candidate being supported is against this and believes in a strong America that will come from strength and security.",
        "B": "Visual imagery of toy army men and models is used to depict the opposing candidates plans to reduce military funding, which, a senator claims would undermine national security. The candidate, portrayed alongside military officers as \"Hail to the Chief\" plays, emphasizes a commitment to negotiating peace from a position of strength. ",
        "selection": [
            1,
            3
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/vSwQjWJIljc\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 137 10462",
        "url": "https://www.youtube.com/watch?v=wE5VHRf7y7U",
        "A": "The candidate discusses the issue of busing students with regard to the racial desegregation of schools. No one wants their child to have an inferior education. He outlines his stance: busing when used only to achieve racial balance is wrong, whereas busing to achieve a better education is acceptable.",
        "B": "The candidate discuses busing from an office desk, emphasizing that no parents desire an inferior education for their child. The candidate asserts that busing should not be used to achieve racial balance but to ensure access to better schools for all children.  ",
        "selection": [
            1,
            2
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/wE5VHRf7y7U\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 210 12268",
        "url": "https://www.youtube.com/watch?v=wXOVxtu_teA",
        "A": "Directly addressing the viewer, the candidate critiques the opposing party's TV commercials claiming high employment, economic health, and effective leadership, contrasting with their own experience. Highlighting the reality of persisting negative circumstances, the candidate calls for collaborative efforts to enact change under his leadership. ",
        "B": "The candidate discusses the republicans who say the economy is healthy, employment is high, and our leadership is great. The candidate argues that what he sees is not this and that the reality is far from what the republicans say. But with him, the country can work together to fix this.",
        "selection": [
            3,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/wXOVxtu_teA\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 2 28",
        "url": "https://www.youtube.com/watch?v=xsJvMNSbxcI",
        "A": "A woman sings throughout the advertisement in opposition of three candidates. She humorously explains that their policies are unacceptable for anyone. Electing such men, she notes, will create regrettable results for the country. The advertisement aims to entertain the audience with vague, snide remarks about the candidates.",
        "B": "A formally dressed women mocks three opposing candidates for their perceived incompetence in song, speaking directly to the audience. The song adopts a light-hearted tone as it uses humor to critique the candidates' shortcomings and faults. ",
        "selection": [
            3,
            2
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/xsJvMNSbxcI\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    },
    {
        "title": "P 147 10624 wm",
        "url": "https://www.youtube.com/watch?v=ysu1KM7ZPds",
        "A": "The advertisement begins at a rally of the opposing candidate. The candidate explains that he was a supporter at that rally, but the opposing candidate has not followed through on his promises since being in office. The candidate, however, would keep his promises if elected, and Americans would win.",
        "B": "A convention hall is depicted, initially filled with cheering supporters of the opposing candidate, then shown empty. The candidate appears and acknowledges past support for the opposing candidate but criticizes their failure to fulfill promises during their administration. Emphasizing a commitment to keeping promises, the candidate seeks support from viewers. ",
        "selection": [
            2,
            1
        ],
        "embed": "<iframe class=\"youtube-video\"  src=\"https://www.youtube.com/embed/ysu1KM7ZPds\" frameborder=\"0\" allow=\"accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture\" allowfullscreen></iframe>"
    }
]

In [2]:
titles = [video["title"] for video in video_data]

# Print titles, one on each line
for title in titles:
    print(title)

P 1500 75016
P 1436 67065
P 1425 66916
P 2116 141528
P 1905 108595
P 2098 140422
P 1415 68241
P 1905 108583
P 2125 142151
P 1353 63367
P 1908 108808
P 1617 81835
P 1817 102732
P 1436 67079
P 1619 81865
P 1708 90169
P 1907 108725
P 1903 108459
P 1618 81849
P 1708 90180
P 2123 141983
P 1425 66913
P 2015 136329
P 2015 136314
P 1615 81773
P 1908 108788
P 1500 75019
P 1691 86717
P 1436 67087
P 2060 138579
P 1581 79623
P 1500 75016
P 2114 141382
P 1616 82778
P 1818 102774
P 1581 79612
P 1816 102673
P 1817 102725
P 1361 63305
P 1631 83508
P 1827 103450
P 1498 74079
P 1840 104205
P 1411 68113
P 1772 101721
P 1415 68229
P 1631 82797
P 1498 74101
P 1190 35257
P 358 9904
P 507 94878
P 374 30448
P 475 96893
P 1197 52122
P 1154 50000
P 1182 51824
P 359 9968
P 500 94375
P 500 94360
P 1051 43133
P 1051 43126
P 1009 40497
P 1009 40490
P 387 30797
P 1232 57581
P 1064 43975
P 316 8988
P 1033 42062
P 1012 40696
P 1289 59973
P 601 114472
P 1190 35259
P 380 30565
P 475 96879
P 1232 57589
P 1085 45276
P 603